In [35]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

# try:
#     driver = webdriver.Chrome(f'91/chromedriver')   
# except:
#     chromedriver_autoinstaller.install(True)
#     driver = webdriver.Chrome(f'91/chromedriver')

driver = webdriver.Chrome(f'../chromedriver')



In [36]:
driver.get("https://v4.map.naver.com")
driver.find_elements_by_css_selector("button.btn_close")[1].click()
# spmc_zoomout
driver.implicitly_wait(10)
#         a:nth-child(3)#

search_box = driver.find_element_by_css_selector("input#search-input")
search_box.send_keys("서울역")

search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)
search_box.clear()

#지도레벨 축소
# for i in range(3):    
#     driver.find_elements_by_css_selector("a.spmc_zoomout")[0].click()
#     time.sleep(0.25)


#print(chrome_ver)

## store ids 
store_ids = [];

In [24]:
store_id_path = 'store_id.txt'

def get_store_id(webl_obj) :
    for store in webl_obj:
        str_id = store.get_attribute("class") + ""
        store_ids.append(str_id.split(',')[1]);
        print(str_id.split(',')[1])
        #WriteTxtFiles(store_id_path, str_id.split(',')[1])


def WriteTxtFiles(path, store_id):    
    with open(path, "a") as file:
        file.write(store_id + ",")
        file.close()



In [37]:
search_query  = ['식당']
search_box = driver.find_element_by_css_selector("input#search-input")
search_box.clear()
current_page_num = 1;
max_search_page_num = 50;

search_box.send_keys(search_query)
#         a:nth-child(3)#
search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)

# 우선 store_id는 100개만 수집
for i in range(max_search_page_num):
    next_button = driver.find_element_by_css_selector("div.paginate").find_elements_by_tag_name("a")    
    stores = driver.find_elements_by_css_selector("ul.lst_site>li")
    get_store_id(stores)
    current_page_num += 1;
    
    for nextItem in next_button:
        if(nextItem.text == "다음"):
            nextItem.click()
            time.sleep(1)
            break
            
        if(nextItem.text == str(current_page_num)):
            nextItem.click()
            time.sleep(1)
            break

store_id_set = set(store_ids)
for store_id in store_id_set:
    WriteTxtFiles(store_id_path, store_id)

37402355
1327043034
1227621180
36166703
36379024
37641712
1331806777
34556586
145365549
1319862091
13151383
32762172
32786158
36015662
11700489
1865411597
13586350
13575201
19774046
1423886812
1411095343
1060660020
667716278
21427490
32520977
37402239
1166904528
36192525
1973129737
1340801285
18137751
1823752520
13446068
36432841
1245603197
36314854
38274444
178835958
1414413246
1847706138
72255050
1583598574
1773981452
1743165101
32872887
1625627918
34806737
1090397990
18132250
37384008
38233893
20521260
18712923
1704928010
13458464
13160676
1866836918
32594093
1725880081
1655072412
11600155
301004844
284951154
457840624
1555137184
1085431656
1732861163
36639853
36628762
36132502
1120274637
1898177359
38466708
13349606
36208670
1052789166
1030168902
38404629
11620882
11727374
965428244
13483839
1261421537
483963111
1607969391
19795126
19882431
1058549910
20034529
1477472215
19878316
1044477499
31427501
34975372
36175074
1339812209
1052201746
34519105
1737815690
1417363078
34129895
129

In [38]:
# 수집된 store_id로 방문자 리뷰 데이터를 수집한다
from bs4 import BeautifulSoup
import requests

with open("store_id.txt", "r") as file:
    store_ids = file.read()
    file.close()
    
driver = webdriver.Chrome(f'../chromedriver')

for store_id in store_ids.split(","):
    if(store_id):
        print("================")
        
        url = f'https://pcmap.place.naver.com/restaurant/{store_id}/review/visitor'
        htm = driver.get(url)
        time.sleep(1)
        
        try:
            for i in range(20):
                more_btn = driver.find_element_by_class_name("_3iTUo")
                if(more_btn):
                    more_btn.click()
                    time.sleep(0.5)
                else:
                    break;
        except Exception as ex:
            print("error catch : " , ex)
        
        print("id : " + store_id)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # li _2Cv-r -> 부모 컨테이너 // span _2tObC  ->별점 // span WoYOw ->리뷰 내용

        for ll_item in soup.find_all('li',{'class' : '_2Cv-r'}):

            rating_ll = ll_item.find('span',{'class' : '_2tObC'})
            review_ll = ll_item.find('span',{'class' : 'WoYOw'})

            if(review_ll):
                #10자 이상 리뷰만 의미있는 값으로 본다.
                if(len(review_ll.text)>10):
                    #3점 ~ 4점대 리뷰는 거른다 (3점미만 : 부정 , 5점 : 긍정)
                    if(float(rating_ll.text) < 3 or float(rating_ll.text) > 4.5):
                        review_content = review_ll.text
                        label = "0" #0 : 긍정, 1 부정
                        if("\n" in review_ll.text):
                            review_content = review_ll.text.replace("\n"," ")
                            
                        #3점 미만이면 부정평가
                        if(float(rating_ll.text) < 3 ):
                            label = "1";
                            
                            with open("review_data.txt", "a") as file:
                                # id , 별점, 리뷰내용, 레이블(0:긍정, 1:부정)
                                file.write(store_id+"\t" + rating_ll.text+"\t" + review_content +"\t"+ label+ "\n") 
    #                     print("rating : " + rating_ll.text)
    #                     print("review : " + review_ll.text)
        file.close()
#        driver.close()
        

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 59335525
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 12046743
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 11560720
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 11671902
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1776488654
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id :

id : 1721171187
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1993052197
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 17816051
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1318994225
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1955885938
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 31461216
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 32218062
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1187045070
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 13167981
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 17722919
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1085678105
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 17722567
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1358314310
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 32217912
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1728010805
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 17815925
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id

id : 11528334
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 34268835
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 17719099
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 15117663
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 38249029
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 888881773
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.44

...

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1038784243
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 34714133
id : 1436711710
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1518974488
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1020760108
id : 1063954725
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1548387355
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Ses

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 13290926
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 19882431
id : 1865411597
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 36208670
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1236436838
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 457840624
id : 37402239
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session in

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1027978015
id : 1539185460
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 372557663
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 31357362
id : 32434727
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 18718329
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 18137751
error catch :  Message: stale element reference: element is not attached to the page document
  (Session info: chrome=91.0.4472

id : 19878316
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 32960167
id : 1090397990
id : 18138022
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1705575287
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 1066747319
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 31259151
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.106)

id : 36798029
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"

KeyboardInterrupt: 